# Data Capture Notebook
This notebook captures sensor data from the Arduino via serial commands.

# Install Required Packages
Install pyserial for serial communication.

In [8]:
!pip install pyserial

  Using cached pyserial-3.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Libraries
Import serial for Arduino communication and time for delays.

In [1]:
# Import required libraries
import serial
import time

# Configure Parameters
Set charge time, measure time, and output filename.

In [27]:
# Set your parameters here
charge_time = 120  # seconds (Arduino charges for 5s, adjust if needed)
measure_time = 3600  # seconds
filename = 'sensor_data_clamp_ultraStart_run1.csv'  # name for the CSV file
port = 'COM8'  # your serial port

# Setup Serial Connection
Initialize serial port for communication with Arduino.

In [28]:
# Initialize serial connection to Arduino
ser = serial.Serial(port, 115200, timeout=1)

# Run Data Capture
Execute the measurement sequence and save data to CSV with progress bar.

In [29]:
# Main data capture loop: send command, read serial, save to CSV, show progress
with open(filename, 'w') as f:
    f.write('t,measureStartTime,full,gain,int_idx,state\n')
    
    # Send start with charge and measure times
    cmd = f"start {charge_time} {measure_time}\n"
    ser.write(cmd.encode())
    print(f"Sent: {cmd.strip()}")
    
    total_time = charge_time + measure_time
    start_time = time.time()
    last_print = start_time
    bar_length = 30
    
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            f.write(line + '\n')
            f.flush()
            if 'DONE' in line:
                # Final progress bar
                percent = 100
                filled = bar_length
                bar = '=' * filled
                print(f"[{bar}] {percent:.0f}% (0 sec remaining)")
                print("Measurement done")
                break
        current_time = time.time()
        if current_time - last_print >= 1:  # Update every second for smoother bar
            elapsed = current_time - start_time
            percent = min(100, (elapsed / total_time) * 100)
            remaining = max(0, total_time - elapsed)
            filled = int(bar_length * percent / 100)
            bar = '=' * filled + ' ' * (bar_length - filled)
            print(f"[{bar}] {percent:.0f}% ({remaining:.0f} sec remaining)", end='\r')
            last_print = current_time

Sent: start 120 3600
[==============================] 100% (0 sec remaining)))
Measurement done
